In [1]:
import pandas as pd
from zipfile import ZipFile
import xml.etree.ElementTree as ET
import os

In [2]:
# Caminho dos arquivos extraidos do kaggle
path_dataset = r'files\classificao-de-notcias.zip'
path_db = r'db'

# Caminho dos arquivos que serão utilizados para a atividade
path_train = r'db\arquivos_competicao\arquivos_competicao\train.csv'
path_test = r'db\arquivos_competicao\arquivos_competicao\test.csv'
path_news = r'db\arquivos_competicao\arquivos_competicao\news'


1. Extraindo os dados do arquivo .zip, baixado do kaggle

In [3]:
def unzip(path, pathFolder):

    # descompacta a base de dados de notícias
    z = ZipFile(path, 'r')

    if os.path.isdir(pathFolder):
        z.extractall(pathFolder)
        z.close()
    else:
        os.mkdir(pathFolder)
        z.extractall(pathFolder)
        z.close()

    print("Arquivo descompactado com sucesso!")
    
# Antes de descompactar os arquivos valida se ja foram descompactados antes
if not os.path.isdir(path_news):
    unzip(path_dataset, path_db)
else:
    print("Arquivo já descompactado")

Arquivo já descompactado


2. Lendo o arquivo de treino como um DataFrame

In [4]:
df_train = pd.read_csv(path_train)
df_train = df_train.sort_values(['ID'])
df_train.head()

,ID,Class
0,news_00002.xml,Mercados
1,news_00003.xml,Mercados
2,news_00006.xml,Mercados
3,news_00007.xml,Economia
4,news_00008.xml,Mercados


3. Criando um df com os textos e titulos extraidos do XML

In [8]:
def extract_xml_text(path_xml):
    tree = ET.parse(path_xml)
    root = tree.getroot()
    headline = root.find('headline').text if root.find('headline') is not None else ''
    paragraphs = root.findall('.//p')
    text = ' '.join([p.text for p in paragraphs if p.text is not None])

    return headline, text

for idx in df_train.index:
    file = df_train.at[idx, 'ID']
    path_xml = f"{path_news}\{file}"
    titulo, texto = extract_xml_text(path_xml)
    df_train.at[idx, 'TITULO'] = titulo
    df_train.at[idx, 'TEXTO'] = texto

df_train = df_train[['ID', 'TITULO', 'TEXTO', 'Class']]
df_train.head()

,ID,TITULO,TEXTO,Class
0,news_00002.xml,"FUTURO OT DEZ/96 CAI PARA 102,23 CONTRA 102,5...","*Futuro (BDPOTZ6)Dez/96 102,23 vs 102,52 no f...",Mercados
1,news_00003.xml,RESUMO TRANSACÇÕES NO MERCADO CONTÍNUO.,"LISBOA, 30 Set (Reuter) - Transacções no ...",Mercados
2,news_00006.xml,"INDICE PSI20 SOBE 1,28 PONTOS PARA 4.764,93.","LISBOA, 24 Out (Reuter) - O índice PSI20 subi...",Mercados
3,news_00007.xml,ÍNDICE PREÇOS PRODUÇÃO INDUSTRIAL AGO 1996 SO...,"LISBOA, 29 Out (Reuter) - O Índice de Pre...",Economia
4,news_00008.xml,BDP INTERROMPIDA POR PROBLEMAS COMUNICAÇÕES.,"LISBOA, 30 Set (Reuter) - A negociação na Bol...",Mercados
